# Week 4 : Lecture 1 (18 September, 2018)

- Binary Search Trees: Inductively
- My first interpreter for arithmetic.
- If time permits: Big Step Operational Semantics.

In [1]:
sealed trait NumTree
case object Leaf extends NumTree
case class Node(n: Int, left: NumTree, right: NumTree) extends NumTree

defined trait NumTree
defined object Leaf
defined class Node

In [2]:
val t1 = Leaf
val t2 = Node(10, Leaf, Leaf)
val t3 = Node(10, Node(8, Leaf, Leaf), Node(15, Leaf, Node(23, Leaf, Leaf)))

t1: Leaf.type = Leaf
t2: Node = Node(10,Leaf,Leaf)
t3: Node = Node(10,Node(8,Leaf,Leaf),Node(15,Leaf,Node(23,Leaf,Leaf)))

In [4]:
def depth(t: NumTree): Int = t match {
    case Leaf => 0
    case Node(_, lChild, rChild) => 1 + math.max(depth(lChild), depth(rChild))
}

defined function depth

In [6]:
depth(t1)
depth(t3)

res5_0: Int = 0
res5_1: Int = 3

In [ ]:
def areAllNodesInATreeLessThan(t: NumTree, k: Int): Boolean = t match {
    case Leaf => true
    case Node(j, left, right) => (j < k) && 
                               areAllNodesInATreeLessThan(left, k) && 
                             areAllNodesInATreeLessThan(right, k)
    
}

In [7]:
def checkForAll(t: NumTree, p: Int => Boolean): Boolean = t match {
    case Leaf => true
    case Node(j, left, right) => p(j) && checkForAll(left, p) && checkForAll(right, p)
}

defined function checkForAll

In [8]:
def isBST(t: NumTree): Boolean = t match {
    case Leaf => true
    case Node(k, lChild, rChild) => {
        def ltk(j: Int): Boolean = j < k 
        def gtk(j: Int): Boolean = j > k
        isBST(lChild) &&
        isBST(rChild) &&
        checkForAll(lChild, ltk) &&
        checkForAll(rChild, gtk)
    }
}


defined function isBST

In [9]:
val t3 = Node(10, Node(8, Leaf, Leaf), Node(15, Leaf, Node(23, Leaf, Leaf)))
val t4 = Node(10, t3, t3)

t3: Node = Node(10,Node(8,Leaf,Leaf),Node(15,Leaf,Node(23,Leaf,Leaf)))
t4: Node = Node(10,Node(10,Node(8,Leaf,Leaf),Node(15,Leaf,Node(23,Leaf,Leaf))),Node(10,Node(8,Leaf,Leaf),Node(15,Leaf,Node(23,Leaf,Leaf))))

In [10]:
sealed trait Expr
case class Const(f: Double) extends Expr 
// 1. We cheated a bit and allowed all floating point numbers
// Also, this deviates from the grammar
case class Ident(s: String) extends Expr
// 2. We allow any string to be an identifier for now instead of the regular expression shown in the grammar.
case class Plus( e1: Expr, e2: Expr ) extends Expr // e1 + e2
case class Minus(e1: Expr, e2: Expr) extends Expr // e1 - e2
case class Mult(e1: Expr, e2: Expr) extends Expr // e1 * e2
case class Div(e1: Expr, e2: Expr) extends Expr // e1 / e2
case class Log(e: Expr) extends Expr
case class Exp(e: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Log
defined class Exp
defined class Sine
defined class Cosine

In [11]:
val e1 = Div(Plus(Ident("x"), Ident("y")), Ident("z"))

e1: Div = Div(Plus(Ident(x),Ident(y)),Ident(z))

In [13]:
def collectAllVars(e: Expr): Set[String] = e match {
    case Const(f) => Set()
    case Ident(id) => Set(id)
    case Plus(e1, e2)  => {
        (collectAllVars(e1)) union (collectAllVars(e2))
    }
    
    case Minus(e1, e2) => {
        (collectAllVars(e1)) union (collectAllVars(e2))
    }
    
    case Mult(e1, e2) => {
        (collectAllVars(e1)) union (collectAllVars (e2))
    }
    
    case Div(e1, e2) => {
        (collectAllVars (e1)) union (collectAllVars (e2))
    }
    
    case Log(e) => { collectAllVars(e) }
    
    case Exp(e) => { collectAllVars(e) }
    case Sine(e) => { collectAllVars(e) }
    case Cosine(e) => { collectAllVars(e) }
}

defined function collectAllVars

In [14]:
collectAllVars(e1)

res13: Set[String] = Set("x", "y", "z")

In [15]:
def prettyPrint(e: Expr): String = e match {
    case Const(f) => s"$f"
    case Ident(id) => id
    case Plus(e1, e2)  => {
        "("+prettyPrint(e1)+" + "+prettyPrint(e2)+")"
    }
    
    case Minus(e1, e2) => {
       "("+prettyPrint(e1)+" - "+prettyPrint(e2)+")"
    }
    
    case Mult(e1, e2) => {
        "("+prettyPrint(e1)+" * "+prettyPrint(e2)+")"
    }
    
    case Div(e1, e2) => {
        "("+prettyPrint(e1)+" / "+prettyPrint(e2)+")"
    }
    
    case Log(e) => { "log("+prettyPrint(e)+")"}
    
    case Exp(e) => { "e^("+prettyPrint(e)+")" }
    case Sine(e) => {  "sin("+prettyPrint(e)+")"}
    case Cosine(e) => { "cos("+prettyPrint(e)+")"}
    
}

defined function prettyPrint

In [16]:
prettyPrint(e1)

res15: String = "((x + y) / z)"

In [17]:
val myEnvironment: Map[String, Double] = Map("x" -> 2.0, "y" -> 1.5, "z" -> 2.8)

myEnvironment: Map[String, Double] = Map("x" -> 2.0, "y" -> 1.5, "z" -> 2.8)

In [18]:
def evalExpr(e: Expr, env: Map[String, Double]): Double = e match {
    case Const(f) => f
    case Ident(s) => {
        if (env.contains(s)) { 
            return env(s)
        } else {
            throw new IllegalArgumentException(s"Define $s please.")
        }
    }
    
    case Plus(e1, e2) => {
        val c1 = evalExpr(e1, env)
        val c2 = evalExpr(e2, env)
        c1 + c2
    }
    case Minus(e1, e2) => {
        val c1 = evalExpr(e1, env)
        val c2 = evalExpr(e2, env)
        c1 - c2
    }
    case Mult(e1, e2) => {
        val c1 = evalExpr(e1, env)
        val c2 = evalExpr(e2, env)
        c1 * c2
    }
    
    case Div(e1, e2) => {
        val c1 = evalExpr(e1, env)
        val c2 = evalExpr(e2, env)
        if (c2 != 0.0) {
            c1 / c2
        } else {
           throw new IllegalArgumentException("Division by zero happened") 
        }
    }
    
    case Log(e)=> {
        val c = evalExpr(e, env)
        if (c > 0.0) {
            math.log(c)
        } else {
            throw new IllegalArgumentException("Log of a negative number happened")
        }
    }
    
    case Exp(e) => {
        val c = evalExpr(e, env)
        math.exp(c) 
        
    }
    
    case Sine(e) => {
        val c = evalExpr(e, env)
        math.sin(c) 
        
    }
    
    case Cosine(e) => {
        val c = evalExpr(e, env)
        math.cos(c) 
    }
}

defined function evalExpr

In [19]:
val j1 = evalExpr(e1, myEnvironment)

j1: Double = 1.25